# FIRE 

This notebook will try to answer the following question:

- When can I stop working and live off my investments? :-)

In [2]:
import sys
from pathlib import Path

project_root = Path.cwd().parent 
src_path = project_root / 'src'

sys.path.append(str(src_path))

In [3]:
import datetime
from datetime import date, timedelta
from finsim.properties import InvestmentProperty
from finsim.simulations import FireSimulation, run_fire_simulation
import pandas as pd
import plotly.express as px
from decimal import Decimal


In [4]:
from decimal import Decimal


init = FireSimulation(
    monthly_income=Decimal("20_000"),
    monthly_expenses=Decimal("16_000"),
    stock_investments=Decimal("100_000"),
    bonds_investments=Decimal("100_000"),
    cash=Decimal("50_000"),
    investment_properties=[
        InvestmentProperty(
            mortgage_left=Decimal("292000"),
            mortgage_rate=Decimal("7.66"),
            mortgage_months=202,
            market_value=Decimal("520000"),
            monthly_income=Decimal("2100"),
            annual_rent_increase_rate=Decimal("0.01"),
        ),
    ],
    stock_return_rate=Decimal("0.06"),
    bonds_return_rate=Decimal("0.02"),
    annual_inflation_rate=Decimal("0.03"),
    annual_income_increase_rate=Decimal("0.03"),
    annual_property_appreciation_rate=Decimal("0.02"),
    invest_cash_surplus=True,
    invest_cash_threshold=Decimal("50_000"),
    invest_cash_surplus_strategy='80-20',
    date=datetime.datetime.fromisoformat('2024-04-01')
)

# simulate for next X years
current_age = 39
expected_age = 85
years = expected_age - current_age
print(f"Simulating for {years} years")
date_of_death = datetime.datetime.now() + timedelta(days=years*365)
print(f"Expected date of death: {date_of_death}")

expected_number_of_months = years * 12
print(f"Expected number of months: {expected_number_of_months}")

simulation, nmb_of_sims = run_fire_simulation(init, expected_number_of_months=expected_number_of_months)
print(f"Number of simulations: {nmb_of_sims}")



Simulating for 46 years
Expected date of death: 2070-03-26 17:51:19.772326
Expected number of months: 552
Number of simulations: 304


In [5]:

df = pd.DataFrame([s.to_dict() for s in simulation])


In [6]:
# find the first month where the monthly income is 0
first_month_with_zero_income = df[df['monthly_income'] == 0].iloc[0]
print(f"First month with zero income: {first_month_with_zero_income['date']}")


First month with zero income: 2049-10-01 00:00:00


In [7]:
# I want to use plotly to create a nice stacked bar chart of bonds and stock investments
# with a line chart showing monthly expenses

fig = px.bar(df, x=df['date'], y=[
        "properties_net_cash_value",
        "stock_investments",
        "bonds_investments",
        "cash",
    ], title='When can you stop working chart')
fig.add_scatter(x=df['date'], y=df['monthly_expenses'], mode='lines', name='Monthly expenses')
fig.add_scatter(x=df['date'], y=df['wealth_inc_properties'], mode='lines', name='Wealth with net properties')

fig.show()
